<a href="https://colab.research.google.com/github/baranceanuvlad/Advanced-Topics-in-Neural-Networks-Template-2023/blob/main/Lab02/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorboard
import math
import random

In [80]:
x = torch.rand(20, 784).to('cuda')
W = torch.rand(784,10).to('cuda')
b = torch.rand(1, 10).to('cuda')
learning_rate = 0.1
y = torch.zeros((20, 10)).to('cuda')

for result in y:
  nr = random.randint(0, 9)
  result[nr] = 1

In [81]:
def sigmoid_function(z):
  result = []
  for value in z:
    result.append( 1.0 / (1.0 + math.e ** (0 - value)) )
  return torch.tensor(result).to('cuda')

In [87]:
def train_perceptron(x, W, b, learning_rate, y_true):
  for _ in range(100):
    for idx, input in enumerate(x):
      z = input.reshape(1, 784) @ W + b
      predicted_y = sigmoid_function(z.flatten())
      error = y_true[idx] - predicted_y
      W = W + learning_rate * input.reshape(784, 1) @ error.reshape(1, 10)
      b = b + learning_rate * error.reshape(1, 10)
  return W, b

In [88]:
train_perceptron(x, W, b, learning_rate, y)

(tensor([[ 0.6141, -0.1278,  0.5730,  ...,  0.5899, -0.4665, -0.5380],
         [ 0.2935,  0.5062, -0.5311,  ...,  0.1693,  0.2624,  0.0489],
         [-0.2771,  0.1114, -0.3807,  ..., -0.0628, -0.5113, -0.0551],
         ...,
         [-0.3820,  0.1959,  0.4267,  ..., -0.4212,  0.2450,  0.1157],
         [-0.1240, -0.4469,  0.2342,  ..., -0.0477,  0.1162, -0.4355],
         [-0.1190, -0.2053,  0.3412,  ...,  0.1228, -0.5104,  0.3821]],
        device='cuda:0'),
 tensor([[-0.0381, -0.7002, -0.3111, -0.3517, -0.4640, -0.6009, -0.6949, -0.9920,
          -0.2791, -0.5407]], device='cuda:0'))